In [13]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import scipy.stats
from scipy.optimize import minimize
from scipy.stats import multivariate_normal
from mpl_toolkits.mplot3d import Axes3D
import operator
from scipy.optimize import minimize
import math

In [290]:
#initialize plotting canvas
%matplotlib tk 
fig = plt.figure( num=1, clear=True)
#seed
seed_gen=np.random.uniform(0,1000,1).astype(int)
np.random.seed(seed=seed_gen)

In [174]:
#generate one path
def gen_path(x_0in,N_in,T_in, b_in, sig_in):
    #T  time limit
    # N #steps
    dt = T_in/N_in 
    x = np.zeros(N_in) #container
    #generate normal distributed 
    dW=np.random.normal(0, 1, N_in)
    #initialize
    # x_0 
    x[0]=x_0in
    for i in range(1,N_in): 
        x[i] =x[i-1] + b_in*dt + sig_in*np.sqrt(dt)*dW[i]
    return(x)

In [16]:
# the liklihood function of the gaussian transition densities
def dist_b(param_b,X,sig_in,T_in): #X are pairs of x_i and x_{i-1}
    f=0
    dt=T_in/len(X)
    sig=10#set sigma
    b=param_b
    for i in range(1,len(X)):
        f=f -(X[i]-X[i-1] - b*dt)**2/(2*dt*(sig)**2) - np.log(np.sqrt(2*math.pi*dt)*sig)
    return(-1*f)

In [17]:
# the liklihood function of the gaussian transition densities
def dist_sig(param_sig,X,b_in,T_in): #X are pairs of x_i and x_{i-1}
    f=0
    dt=T_in/len(X)
    b=b_in # set b
    sig=param_sig
    for i in range(1,len(X)):
        f=f -(X[i]-X[i-1] - b*dt)**2/(2*dt*(sig)**2) - np.log(np.sqrt(2*math.pi*dt)*sig) 
    return(-1*f)

In [18]:
# the liklihood function of the gaussian transition densities
def dist_2d(param_2d,X, T_in): #X are pairs of x_i and x_{i-1}
    f=0
    dt=T_in/len(X)
    sig=param_2d[1]
    b=param_2d[0]
    
    #gamma prior
    
    a=2
    loc=9
    scale=3
    
    for i in range(1,len(X)):
        f=f -(X[i]-X[i-1] - b*dt)**2/(2*dt*(sig)**2) - np.log(np.sqrt(2*math.pi*dt)*sig) 
            #+np.log(scipy.stats.gamma.pdf(sig, a, loc, scale))
    return(-1*f)

In [30]:
#path set parameters
N=10000
#sig_gen = np.zeros(N);
#b_gen = np.zeros(N);
sig_gen=17
b_gen=165
x_0=0
T=1
dt=T/N

data=gen_path(x_0in=x_0,N_in=N,T_in=T, b_in=b_gen, sig_in=sig_gen)


In [31]:
ax1 = fig.add_subplot(221)
ax1.cla()
ax1.plot(data);

/home/alhaddwt/anaconda3/lib/python3.7/site-packages/matplotlib/figure.py:98: MatplotlibDeprecationWarning: 
Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  "Adding an axes using the same arguments as a previous axes "


In [32]:
int_mean=np.random.uniform(-1000,1000,1)
T=1
sig_op=sig_gen
args=(data,sig_op,T)
x0=int_mean
min_param=minimize(dist_b,x0,args, tol=1e-15);min_param

      fun: 882.9259847106307
 hess_inv: array([[100.00008]])
      jac: array([-7.62939453e-06])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 179
      nit: 4
     njev: 56
   status: 2
  success: False
        x: array([171.35211377])

In [33]:
int_sig=np.random.uniform(0,100,1)
T=1
b_op=b_gen
args=(data,b_op ,T)
x0=int_sig
bnds = np.array([(0.001,5000)])
min_param=minimize(dist_sig,x0,args,bounds=bnds, tol=1e-12);min_param

      fun: array([-3437.38380919])
 hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>
      jac: array([0.10682015])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 56
      nit: 5
   status: 0
  success: True
        x: array([17.15774893])

In [196]:
int_param=np.random.uniform(0,50,2) #np.array(  [3,1]  , dtype=float);
T=1
args=(data,T)
x0=int_param
#bnds = ((10, 10), (500, 25))
#min_param=minimize(dist_2d,x0,args,bounds=bnds, tol=1e-12);min_param

min_param=minimize(dist_2d,x0,args, tol=1e-12);min_param

/home/alhaddwt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in log
  from ipykernel import kernelapp as app


      fun: -3437.45233300633
 hess_inv: array([[4.17298706e+02, 3.01028456e-01],
       [3.01028456e-01, 1.32153851e-02]])
      jac: array([0.00015259, 0.05099487])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 856
      nit: 16
     njev: 211
   status: 2
  success: False
        x: array([171.05868447,  17.15791344])

In [200]:
real_param=np.array([b_gen,sig_gen ], dtype=float)
min_param.x - real_param

array([6.05868447, 0.15791344])

In [35]:
x, y = np.mgrid[-200:200:10, 1:25:1]
pos = np.empty(x.shape + (2,))
pos[:, :, 0] = x; pos[:, :, 1] = y

In [36]:
value_pos=np.empty((pos.shape[0],pos.shape[1] ))
for i in range(0,pos.shape[0]):
    for j in range(0,pos.shape[1]):
        value_pos[i,j]=dist_2d(pos[i,j,:],data,T_in=1)

In [37]:
ax2 = fig.add_subplot(212, projection='3d')
ax2.cla()
ax2.plot_surface(x, y, value_pos, cmap=cm.coolwarm,linewidth=0, antialiased=False)
ax2.scatter(min_param.x[0], min_param.x[1], dist_2d(min_param.x,data,T_in=1), marker='^',s=600)

In [366]:
#convergence testing
def convergence_test(x_0=0, b_gen=165,sig_gen=17,T_in=1,N=np.array(  [500,10000]  , dtype=int)):
    int_param=np.array(  [30,10]  , dtype=float) #np.random.uniform(0,50,2)
    real_param=np.array([b_gen,sig_gen ])
    min_results=np.zeros((len(N),2))
    error=np.empty_like(min_results)
    bnds = ((1e-5, None), (1e-5, None))
    
    for i in range(0, len(N)): #generating data each time !! instead of recycling 
        data=gen_path(x_0in=x_0,N_in=N[i],T_in=T_in, b_in=b_gen, sig_in=sig_gen)
        print(N[i])
        args=(data,T_in)
        x0=int_param
        min_param=minimize(dist_2d,x0,args,method='SLSQP',bounds=bnds, tol=1e-24);
        error[i,:]= np.abs(min_param.x - real_param )
        min_results[i:]=min_param.x
    return(min_results, error)
